In [ ]:
import os
import io
import contextlib
from dotenv import load_dotenv

load_dotenv()

In [ ]:
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")
assert GROQ_API_KEY is not None

In [ ]:
BRIGHT_DATA_SERP_API_KEY=os.environ.get("BRIGHT_DATA_SERP_API_KEY")
assert BRIGHT_DATA_SERP_API_KEY is not None

In [ ]:
from langchain_brightdata import BrightDataSERP

serp_tool=BrightDataSERP(
    bright_data_api_key=BRIGHT_DATA_SERP_API_KEY,
    search_engine="google",
    country="us",
    parse_results=True
)

In [ ]:
serp_tool.name


In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [ ]:
agent_prompt="""
You are an expert assistant that can search the internet to find the best reddit communities for any given topic.
You are also an expert at finding niche communities that discuss the same topic.
"""

In [ ]:
from pydantic import BaseModel, Field

class RedditCommunity(BaseModel):
    name: str = Field(description="Formatted name for the Reddit community")
    url: str = Field(description="The complete URL of the Reddit community")
    subreddit_slug: str = Field(description="The slug of the subreddit such as r/python or r/web or r/trending")
    member_count: int | None = Field(description="Current member count, if available")

class RedditCommunities(BaseModel):
    communities: list[RedditCommunity] = Field(description="The list of Reddit communities")


In [ ]:
from langgraph.prebuilt import create_react_agent

reddit_agent = create_react_agent(
    model=model,
    tools=[serp_tool],
    prompt=agent_prompt,
    response_format=RedditCommunities
)

results=reddit_agent.invoke(
    {"messages":[{"role":"user", "content":"python, javascript, django,web"}]},
    stream_mode="values"
)
results['messages'][-1].content